In [3]:
import os
import OpenDartReader
import FinanceDataReader as fdr


api_key='6ac6c88791643c87a1e2e0b33040e9c953a8287d'

dart = OpenDartReader(api_key)
report = dart.finstate("005930", 2018, 11011)
report.head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,...,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord,currency
0,20190401004781,11011,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동자산,...,2018.12.31 현재,"174,697,424,000,000",제 49 기,2017.12.31 현재,"146,982,464,000,000",제 48 기,2016.12.31 현재,"141,429,704,000,000",1,KRW
1,20190401004781,11011,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동자산,...,2018.12.31 현재,"164,659,820,000,000",제 49 기,2017.12.31 현재,"154,769,626,000,000",제 48 기,2016.12.31 현재,"120,744,620,000,000",3,KRW
2,20190401004781,11011,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,자산총계,...,2018.12.31 현재,"339,357,244,000,000",제 49 기,2017.12.31 현재,"301,752,090,000,000",제 48 기,2016.12.31 현재,"262,174,324,000,000",5,KRW
3,20190401004781,11011,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,유동부채,...,2018.12.31 현재,"69,081,510,000,000",제 49 기,2017.12.31 현재,"67,175,114,000,000",제 48 기,2016.12.31 현재,"54,704,095,000,000",7,KRW
4,20190401004781,11011,2018,00126380,005930,CFS,연결재무제표,BS,재무상태표,비유동부채,...,2018.12.31 현재,"22,522,557,000,000",제 49 기,2017.12.31 현재,"20,085,548,000,000",제 48 기,2016.12.31 현재,"14,507,196,000,000",9,KRW


In [4]:
report.columns

Index(['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'stock_code',
       'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_nm',
       'thstrm_dt', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_dt', 'frmtrm_amount',
       'bfefrmtrm_nm', 'bfefrmtrm_dt', 'bfefrmtrm_amount', 'ord', 'currency'],
      dtype='object')

In [2]:
# 종목 목록 가져오기
stock_list = fdr.StockListing('krx').dropna()
stock_list.head(1)

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,67300,1,600,0.9,67100,67300,66700,11334726,759192857640,401766365615000,5969782550,STK


In [3]:
# 숫자로 모두 변환
def convert_str_to_float(value):
    if type(value) == float:
        return value
    elif value == '-':
        return 0
    else:
        return float(value.replace(',',''))

In [5]:
# 데이터 저장하는 함수 선언
def exrtact_info_and_save(dart, path, year, name, code, report_type, report_type_code):
    report = dart.finstate(code, year, report_type_code)
    if report is None:
        pass
    else:
        submission_date = report['rcept_no'].iloc[0][:8]
        report = report[['fs_nm','account_nm','thstrm_amount','sj_nm']]
        report.rename({
              'fs_nm','개별/연결'
            , 'account_nm','계정명'
            , 'sj_nm','재무제표명'
            , 'thstrm_dt','당기일자'
            , 'thstrm_amount','금액'
            
        }, axis=1, inplace=True)
        report['금액'] = report['금액'].apply(convert_str_to_float)
        
        try:
            os.mkdir(path + '/' + name)
        except:
            pass
        print(report)
        report.to_csv(path+"/{}/{}_{}년_{}.csv".format(name,submission_date,year,report_type))

In [6]:
import time
path='./data/'
dart=OpenDartReader(api_key)
for code, name in stock_list[['Code','Name']].values:
    for year in range(2019,2023):
        for report_type, report_type_code in zip(["1분기보고서", "반기보고서", "3분기보고서", "사업보고서"],
                                                 ['11013','11012','11014','11011']):
            print(name, year, report_type)
            while True:
                try:
                    exrtact_info_and_save(  
                                            name=name
                                          , dart=dart
                                          , path=path
                                          , year=year
                                          , code=code
                                          , report_type=report_type
                                          , report_type_code=report_type_code)
                    time.sleep(0.5)
                    break
                except:
                    time.sleep(60)


삼성전자 2019 1분기보고서
